In [1]:
import pandas as pd
import json
from collections import defaultdict

from bokeh.plotting import figure, curdoc, show  # Crear gráficos
from bokeh.io import show, output_file, output_notebook  # Mostrar o guardar gráficos
from bokeh.models import ColumnDataSource, Select, CustomJS  # Fuente de datos y Controles interactivos
from bokeh.layouts import layout, row  # Organizar gráficos y widgets
from bokeh.palettes import Category10

output_notebook()

Loading BokehJS ...

In [2]:
# Ruta relativa al archivo JSON
ruta_json = "../Jsonl/course-creaaa1/course-creaaa1-limpio.json"

In [3]:
# Ejemplo de uso:
codigo_a_nombre = {
    "video_intro": "vKq2NotGPJQ",
    "LR_1_Video1_Semana1": "U3cK1QMIIEQ",
    "LR_1_Video2_Semana1": "9aNQZ9dKXRY",
    "LR_1_Video3_Semana1": "lsNxh-lSpCY",
    "LR_1_Video4_Semana1": "C3LnEvN0qZ0",
    "LR_1_Video5_Semana1": "vbpbkQE5K_Q",
    "LR_1_Video6_Semana1": "zCFa0xjGXGQ",
    "LR_1_Video7_Semana1": "qlS7ShZfb-c",
    "LR_1_Video8_Semana1": "8cKRb9CKtxk",
    "LR_1_Video9_Semana1": "WyrfIZ6VBcM",
    "LR_1_Video10_Semana1": "NgUhK3rw1IE",
    "LR_1_Video11_Semana1": "ttP0EyzSbbo",
    "LR_1_Video12_Semana1": "Vy4FWDyjZo4",
    "LR_2_Video1_Semana1": "leg7NPlfNf0",
    "LR_2_Video2_Semana1": "avTMbQWrFgM",
    "LR_2_Video3_Semana1": "cNoUwGM1DQs",
    "LR_2_Video4_Semana1": "6Mst559v-Uc",
    "LR_2_Video5_Semana1": "CNQpefXv5DY",
    "LR_2_Video_Semana1": "6W1_fBZFqns",
    "LR_1_Video1_Semana2": "o5VwDVJ7N3Q",
    "LR_1_Video2_Semana2": "LluqYlh2xg4",
    "LR_1_Video3_Semana2": "eE658thjDj8",
    "LR_1_Video4_Semana2": "QbEpClHzTeM",
    "LR_1_Video5_Semana2": "MCG0or2ULB4",
    "LR_1_Video6_Semana2": "ol-vGTdHBNU",
    "LR_1_Video7_Semana2": "WTXS0IMQ3Ss",
    "LR_1_Video8_Semana2": "9kqXmM3b3wc",
    "VS1_Video1_Semana2": "_zQHV3vCGpA",
    "VS2_Video2_Semana2": "RropOrUc2AE",
    "Video1_Semana3": "VGHSSIUyFhI",
    "Video1_Semana4": "kyGRuJXaboU",
}

In [4]:
# Lista de interacciones relacionadas con videos
interacciones_video = ["play_video", "pause_video", "seek_video", "stop_video"]

In [5]:
# Leer el archivo JSON
with open(ruta_json, "r", encoding="utf-8") as f:
    data = json.load(f)
    
# Convertir a DataFrame
data = pd.DataFrame(data)

In [6]:
# Función para contar interacciones
def contar_interacciones(data, interacciones):
    conteos = defaultdict(lambda: defaultdict(int))  # Diccionario anidado {interacción: {código_video: conteo}}
    for _, row in data.iterrows():
        if row.get("name") in interacciones:  # Filtrar solo las interacciones definidas en la lista
            try:
                evento_json = json.loads(row.get("event", '{}'))
                codigo_video = evento_json.get('code', '')  # Cambia 'code' si la clave tiene otro nombre
                if codigo_video:
                    conteos[row.get("name")][codigo_video] += 1
            except json.JSONDecodeError as e:
                print(f"Error al decodificar línea: {e}")
    return conteos

In [7]:
# Convertir los códigos de video a nombres y preparar los datos para la gráfica
def convertir_codigos_a_nombres(conteos, codigo_a_nombre):
    """
    Convierte los códigos de video en los nombres correspondientes dentro de un diccionario anidado.
    
    Parameters:
        conteos (defaultdict): Diccionario anidado {interacción: {código_video: conteo}}.
        codigo_a_nombre (dict): Diccionario que mapea códigos a nombres de videos.
    
    Returns:
        defaultdict: Diccionario con los códigos reemplazados por nombres.
    """
    conteos_con_nombres = defaultdict(dict)
    for interaccion, videos in conteos.items():
        for codigo, conteo in videos.items():
            nombre_video = codigo_a_nombre.get(codigo, codigo)  # Si no se encuentra el código, se usa el original
            conteos_con_nombres[interaccion][nombre_video] = conteo
    return conteos_con_nombres


In [8]:
# Obtener los conteos
conteos = contar_interacciones(data, interacciones_video)
# Diccionario con nombres
conteos_con_nombres = convertir_codigos_a_nombres(conteos, codigo_a_nombre)  

In [9]:
# Función para preparar los datos
def preparar_datos(conteos, interaccion_seleccionada):
    """
    Prepara los datos para la gráfica dependiendo de la interacción seleccionada.
    """
    if interaccion_seleccionada == "Todas las interacciones":
        # Preparar datos apilados para todas las interacciones
        data = {"videos": list(codigo_a_nombre.values())}
        for tipo, videos in conteos.items():
            data[tipo] = [videos.get(codigo, 0) for codigo in codigo_a_nombre.keys()]
    else:
        # Datos específicos para una sola interacción
        data = {
            "videos": list(codigo_a_nombre.values()),
            "reproducciones": [
                conteos.get(interaccion_seleccionada, {}).get(codigo, 0)
                for codigo in codigo_a_nombre.keys()
            ],
        }

    return ColumnDataSource(data=data)

In [10]:
# Callback para actualizar la gráfica
def actualizar_grafica(attr, old, new):
    """
    Actualiza la gráfica según la interacción seleccionada.
    """
    nueva_fuente = preparar_datos(conteos, select.value)
    source.data = nueva_fuente.data

    # Actualizar las barras de la gráfica
    if select.value == "Todas las interacciones":
        p.renderers = []  # Limpiar las barras existentes
        p.vbar_stack(
            list(conteos.keys()), 
            x="videos", 
            source=source, 
            width=0.8,
            color=Category10[len(conteos)], 
            legend_label=list(conteos.keys()),
        )
        p.legend.click_policy = "mute"
    else:
        p.renderers = []  # Eliminar las barras existentes
        p.vbar(
            x="videos", 
            top="reproducciones", 
            width=0.8, 
            source=source,
            fill_color="skyblue", 
            line_color="black",
        )

In [11]:
interaccion_inicial = "Todas las interacciones"

# Preparar fuente inicial de datos
source = preparar_datos(conteos, interaccion_inicial)

In [14]:
# Crear gráfica inicial con herramientas personalizadas
p = figure(
    x_range=list(codigo_a_nombre.values()), 
    title="Interacciones por Video",
    x_axis_label="Videos", 
    y_axis_label="Reproducciones",
    width=800, 
    height=400,
    tools="pan,box_zoom,wheel_zoom,save,reset",  # Herramientas personalizadas
    toolbar_location="right",  # Ubicación de la barra de herramientas
)

# Configurar la orientación de las etiquetas del eje X
p.xaxis.major_label_orientation = 0.785  # En radianes, 45 grados

p.vbar_stack(
    list(conteos.keys()), 
    x="videos", 
    source=source, 
    width=0.8,
    color=Category10[len(conteos)], 
    legend_label=list(conteos.keys()),
)
p.legend.click_policy = "mute"

In [15]:
# Crear el widget Select
select = Select(
    title="Seleccionar interacción", 
    value=interaccion_inicial,
    options=["Todas las interacciones"] + list(conteos.keys()),
)
select.on_change("value", actualizar_grafica)

# Layout final (horizontal)
layout_final = row(select, p)  # Organizar de forma horizontal
curdoc().add_root(layout_final)
show(layout_final)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/js_callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

